<a href="https://colab.research.google.com/github/Isaacostam/Data-Science-Em-Producao/blob/main/m02_v01_store_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 IMPORTS

In [1]:
import pandas as pd

##0.1 Helper Functions

## 0.2 Loading data


In [3]:
df_sales_raw = pd.read_csv('Data/train.csv', low_memory=False)

# 1.0 DESCRICAO DOS DADOS


## 1.1

## 1.2

##1.3

## 1.4 

##1.5

##1.6

##1.7


##1.8
